In [1]:
import bql
import bqplot as bqp
import ipydatagrid as ipd
import ipywidgets as widgets
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from datetime import datetime, timedelta
from IPython.display import display, clear_output
from plotly.subplots import make_subplots

In [2]:
# Connect to BQL
bq = bql.Service()

### Import Data from Coverage List

In [3]:
# Define the file path
file_path = 'altd_coverage_list_20250319.xlsx'

muni_pattern = r'^\d.*US Equity$'

# Read the 'coverage_list' sheet, skipping the first 5 rows
coverage_list = pd.read_excel(file_path, sheet_name='ALTD Coverage', skiprows=5).dropna()
muni_removal = ~coverage_list['Ticker'].str.match(muni_pattern, na=False)
coverage_list = coverage_list[muni_removal]
public_coverage = coverage_list[(coverage_list['Ownership'] == 'Public')&(coverage_list['Bloomberg Second Measure'] == 'Y')]
possible_tickers = public_coverage.Ticker.to_list()

# Read the 'brand_mapping' sheet, skipping the first 5 rows
brand_mapping = pd.read_excel(file_path, sheet_name='ALTD Coverage Brands', skiprows=5).dropna()
public_brands = brand_mapping[brand_mapping['Ticker'].isin(possible_tickers)]

In [4]:
public_brands.head()

,Ticker,Company,Brand,Ownership,Domicile,Bloomberg Second Measure,Placer.ai,Similarweb
519,139480 KS Equity,E-MART Inc,Bristol Farms Brand,Public,KR,Y,N,Y
520,139480 KS Equity,E-MART Inc,EMart24,Public,KR,N,N,Y
521,139480 KS Equity,E-MART Inc,Lazy Acres Natural Market,Public,KR,Y,Y,Y
522,139480 KS Equity,E-MART Inc,Metropolitan Market,Public,KR,Y,N,Y
523,139480 KS Equity,E-MART Inc,New Leaf Community Markets,Public,KR,Y,N,Y


### Data Retreival Functions

In [5]:
def get_data(universe, data_items, with_params=None, preferences=None):
    """Requests data from BQL and returns a DataFrame."""
    request = bql.Request(universe, data_items, with_params=with_params, preferences=preferences)
    response = bq.execute(request)
    df = pd.concat([data_item.df() for data_item in response], axis=1)
    return df

In [6]:
def get_transaction_data(universe):
    """Returns quarterly history of observed sales data."""
    #universe = mapping.index.to_list()
    universe = universe
    online = bq.data.observed_sales(
        fa_period_type='Q', 
        fa_period_offset=bq.func.range('-30', '0'),
        channel='Online'
    )
    in_store = bq.data.observed_sales(
        fa_period_type='Q', 
        fa_period_offset=bq.func.range('-30', '0'),
        channel='In_Store'
    )
    
    data_items={
        'Online': online,
        'In-Store': in_store
               }

    transaction_data = get_data(universe, data_items)[['PERIOD_END_DATE','Online','In-Store']].fillna(0)
    transaction_data = transaction_data.loc[:, ~transaction_data.columns.duplicated()]
    
    together_total = (
        transaction_data
        .assign(Total = lambda x: x['Online'] + x['In-Store'])
        .assign(pct_online = lambda x: x['Online'] / x['Total'])
        .assign(pct_in_store = lambda x: 1 - x['pct_online'])
    )

    together_pct = (
        together_total
        .set_index('PERIOD_END_DATE')
        [['Online','In-Store','Total']]
        .pct_change(periods=4)
        .dropna(thresh=2)
        .reset_index()
    )
    
    return together_total, together_pct

#together_total, together_pct = get_transaction_data('NFLX US Equity')

In [7]:
def get_company_data(universe):
    """
    Pulls company specific data before the need to invoke segment level
    information. E.g., Bloomberg Second Measure Y/Y Sales, Days into 
    Quarter, Company Name, BICS Industry etc.
    """
    # Time series of daily transaction data
    bsm_daily_sales_online = (
        bq.data.observed_sales(dates=bq.func.range('-545D', '0D'), channel='Online')
        .dropna()
    )

    bsm_daily_sales_in_store = (
        bq.data.observed_sales(dates=bq.func.range('-545D', '0D'), channel='In_Store')
        .dropna()
    )

    # Reported quarter end date
    current_q_start = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='0'
    )['PERIOD_END_DATE']
    
    # Unreported quarter end date
    current_q_end = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='1'
    )['PERIOD_END_DATE']
    
    # Comparison quarter start date
    comparison_q_start = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='-4'
    )['PERIOD_END_DATE']
    
    # Comparison quarter end date
    comparison_q_end = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='-3'
    )['PERIOD_END_DATE']
    
    # The sum of the daily transactions across the elapsed days in the 
    # current quarter
    current_q_sales_online = (
        bsm_daily_sales_online
        .matches(
            (bsm_daily_sales_online['DATE'] > current_q_start)
            .and_(bsm_daily_sales_online['DATE'] <= current_q_end)
        )
    )

    current_q_sales_in_store = (
        bsm_daily_sales_in_store
        .matches(
            (bsm_daily_sales_in_store['DATE'] > current_q_start)
            .and_(bsm_daily_sales_in_store['DATE'] <= current_q_end)
        )
    )

    total_days_in_current_quarter = current_q_end - current_q_start
    
    # How many days into the current quarter do we have daily transaction 
    # data for, ie., how many days have elapsed since the last quarter end 
    # date, ensuring that if we have passed the expected current quarter end 
    # date, we cap at the total number of days in the quarter
    days_elapsed_in_current_q = bq.func.min(
        bsm_daily_sales_online.last()['DATE'] - current_q_start,
        total_days_in_current_quarter
    )
    
    # Isolate the comparison quarter sales figures to just be between the
    # comparison quarter start and end dates
    comparison_q_sales_online = (
        bsm_daily_sales_online
        .matches(
            (bsm_daily_sales_online['DATE'] > comparison_q_start)
            .and_(bsm_daily_sales_online['DATE'] <= comparison_q_end)
        )
    )

    comparison_q_sales_in_store = (
        bsm_daily_sales_in_store
        .matches(
            (bsm_daily_sales_in_store['DATE'] > comparison_q_start)
            .and_(bsm_daily_sales_in_store['DATE'] <= comparison_q_end)
        )
    )
    
    data_items = {
        'Total Days in Current Quarter': total_days_in_current_quarter,
        'Days Elapsed in Current Quarter': days_elapsed_in_current_q,
        'Days in Comp Quarter': comparison_q_end - comparison_q_start,
        'Current Quarter Online Sales': current_q_sales_online,
        'Current Quarter In-Store Sales': current_q_sales_in_store,
        'Comparison Quarter Online Sales': comparison_q_sales_online,
        'Comparison Quarter In-Store Sales': comparison_q_sales_in_store
    }
    
    with_params = {
        'act_est_mapping': 'precise',
        'filing_status': 'MRC'
    }

    request = bql.Request(universe, data_items, with_params=with_params, preferences=None)
    company_data = bq.execute(request)

    quarter_metadata = pd.concat([company_data[0].df(), company_data[1].df(),company_data[2].df()],axis=1).reset_index(drop=True)

    total_curr = quarter_metadata.loc[0,'Total Days in Current Quarter']
    days_in = quarter_metadata.loc[0,'Days Elapsed in Current Quarter']
    total_comp = quarter_metadata.loc[0,'Days in Comp Quarter']

    curr_q_online = company_data[3].df().reset_index(drop=True)[['PERIOD_END_DATE','Current Quarter Online Sales']]
    curr_q_in_store = company_data[4].df().reset_index(drop=True)[['Current Quarter In-Store Sales']]

    comp_q_online = company_data[5].df().reset_index(drop=True)[['Comparison Quarter Online Sales']]
    comp_q_in_store = company_data[6].df().reset_index(drop=True)[['Comparison Quarter In-Store Sales']]

    if days_in != total_curr:
        comp_q_online = comp_q_online.loc[:len(curr_q_online)-1]
        comp_q_in_store = comp_q_in_store.loc[:len(curr_q_online)-1]

    final_df = (
        pd.concat([curr_q_online,curr_q_in_store,comp_q_online,comp_q_in_store],axis=1)
        .fillna(0)
        .assign(curr_o_cum_sum = lambda x: x['Current Quarter Online Sales'].cumsum())
        .assign(comp_o_cum_sum = lambda x: x['Comparison Quarter Online Sales'].cumsum())
        .assign(qtd_online = lambda x: x['curr_o_cum_sum'] / x['comp_o_cum_sum'] - 1)
        
        .assign(curr_r_cum_sum = lambda x: x['Current Quarter In-Store Sales'].cumsum())
        .assign(comp_r_cum_sum = lambda x: x['Comparison Quarter In-Store Sales'].cumsum())
        .assign(qtd_in_store = lambda x: x['curr_r_cum_sum'] / x['comp_r_cum_sum'] - 1)
    
        .assign(qtd_total = lambda x: (x['curr_r_cum_sum'] +  x['curr_o_cum_sum']) / (x['comp_r_cum_sum'] + x['comp_o_cum_sum']) -1)
    
        .assign(pct_o = lambda x: x['curr_o_cum_sum'] / (x['curr_o_cum_sum'] + x['curr_r_cum_sum']))
        .assign(pct_r = lambda x: 1 - x['pct_o'])
    
        .rename(columns = {'qtd_online':'QTD - Online','qtd_in_store':'QTD - In-Store','qtd_total':'QTD - All','pct_o':'QTD - % Online','pct_r':'QTD - % In-Store'})
    )

    return final_df, quarter_metadata
    
#final_df = get_company_data(universe)

In [12]:
def get_brand_data2(universe, brands=[], brand_breakout=False):
    # Define the date range for the last ~545 days
    date_range = bq.func.range('-545D', '0D')
    
    # Create a dictionary to hold the data items for each brand
    data_items = {}
    if brand_breakout:
        for brand in brands:
            # Escape any single quotes in the brand name
            safe_brand = brand.replace("'", "\\'")
            # Create the observed_sales data item for this brand
            data_items[brand] = bq.data.observed_sales(
                brand=safe_brand,
                dates=date_range
            )
    else:
        brands=['Online','In_Store']
        for brand in brands:
            # Create the observed_sales data item for this brand
            data_items[brand] = bq.data.observed_sales(
                channel=brand,
                dates=date_range
            )
    
    # Create and execute the request for the specified universe and data items
    request = bql.Request(universe, data_items)
    response = bq.execute(request)
    
    # Concatenate the resulting DataFrames side by side
    df = pd.concat([item.df() for item in response], axis=1)
    
    # Remove any duplicated columns
    df = df.loc[:, ~df.columns.duplicated()]
    
    # Drop unnecessary columns (like SOURCE, BRAND, DATE, PER, and CURRENCY)
    columns_to_drop = [col for col in ['SOURCE', 'BRAND','CHANNEL'] if col in df.columns]
    daily_data = df.drop(columns=columns_to_drop).drop(['DATE', 'PER', 'CURRENCY'], axis=1)
    
    # Retrieve quarter metadata (sales_rev_turn returns a dict-like series for each)
    current_q_start = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='0'
    )['PERIOD_END_DATE']
    
    # Unreported quarter end date
    current_q_end = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='1'
    )['PERIOD_END_DATE']
    
    # Comparison quarter start date
    comparison_q_start = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='-4'
    )['PERIOD_END_DATE']
    
    # Comparison quarter end date
    comparison_q_end = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='-3'
    )['PERIOD_END_DATE']

    data_items_meta = {
        'Current Quarter Start': current_q_start,
        'Current Quarter End': current_q_end,
        'Comparison Quarter Start': comparison_q_start,
        'Comparison Quarter End': comparison_q_end,
        'Total Days in Current Quarter': current_q_end - current_q_start,
        'Days in Comp Quarter': comparison_q_end - comparison_q_start
    }
    
    with_params = {
        'act_est_mapping': 'precise',
        'filing_status': 'MRC'
    }
    
    request = bql.Request(universe, data_items_meta, with_params=with_params, preferences=None)
    company_data = bq.execute(request)
    
    quarter_meta = pd.concat([data_item.df() for data_item in company_data], axis=1)

    # Extract date boundaries from quarter_meta (assuming one row)
    curr_start = pd.to_datetime(quarter_meta['Current Quarter Start'].iloc[0])
    curr_end   = pd.to_datetime(quarter_meta['Current Quarter End'].iloc[0])
    comp_start = pd.to_datetime(quarter_meta['Comparison Quarter Start'].iloc[0])
    comp_end   = pd.to_datetime(quarter_meta['Comparison Quarter End'].iloc[0])
    
    # Filter the DataFrame into current and comparison quarter data
    curr_quarter = daily_data[(daily_data['PERIOD_END_DATE'] >= curr_start) & 
                              (daily_data['PERIOD_END_DATE'] <= curr_end)].copy()
    comp_quarter = daily_data[(daily_data['PERIOD_END_DATE'] >= comp_start) & 
                              (daily_data['PERIOD_END_DATE'] <= comp_end)].copy()

    quarter_meta['Days Elapsed in Current Quarter'] = len(curr_quarter)
    
    # Identify numeric columns (the sales/brand data)
    brand_cols = curr_quarter.select_dtypes(include=[np.number]).columns.tolist()
    
    # Compute cumulative sums for each brand (QTD values)
    for col in brand_cols:
        curr_quarter[col] = curr_quarter[col].cumsum()
        comp_quarter[col] = comp_quarter[col].cumsum()
    
    # Reset the index so we can merge on the row number
    curr_quarter = curr_quarter.reset_index(drop=True)
    comp_quarter = comp_quarter.reset_index(drop=True)
    
    # Merge the current and comparison quarters on the index (row number)
    merged = pd.merge(
        curr_quarter, comp_quarter,
        left_index=True, right_index=True,
        suffixes=('_curr', '_comp')
    )
    
    # Compute the QTD year-over-year growth for each brand.
    # For example: Walmart_YoY_QTD = (Walmart_curr / Walmart_comp) - 1
    for brand in brands:
        curr_col = brand + '_curr'
        comp_col = brand + '_comp'
        merged[brand + '_YoY_QTD'] = merged[curr_col] / merged[comp_col] - 1
    
    # Rename the current quarter date column for clarity
    merged.rename(columns={'PERIOD_END_DATE_curr': 'PERIOD_END_DATE'}, inplace=True)
    
    # Set the index to be PERIOD_END_DATE
    merged.set_index('PERIOD_END_DATE', inplace=True)
    
    # Drop all columns that do not end in 'YoY_QTD'
    merged_growth = merged[[col for col in merged.columns if col.endswith('YoY_QTD')]].copy()
    
    # Rename columns to drop the '_YoY_QTD' suffix so they are just the brand names
    merged_growth.rename(columns=lambda x: x.replace('_YoY_QTD', ''), inplace=True)

    share = pd.DataFrame()
    #share of sales
    share['Total'] = curr_quarter.set_index('PERIOD_END_DATE').sum(axis=1)

    for brand in brands:
        share[brand] = curr_quarter.set_index('PERIOD_END_DATE')[brand] / share['Total']
    share.drop(columns=['Total'],inplace=True)
    
    return merged_growth, share, quarter_meta

# merged_growth, share, quarter_meta = get_brand_data2(
#     universe='WMT US Equity', 
#     brands=["Walmart", "Sam's Club", "Walmart Gas", "Shoes.com"],
#     #brand_breakout=True
# )
# share.head()

,Online,In_Store
PERIOD_END_DATE,,
2025-01-31,0.256793,0.743207
2025-02-01,0.244321,0.755679
2025-02-02,0.247036,0.752964
2025-02-03,0.245138,0.754862
2025-02-04,0.244543,0.755457


In [8]:
def get_brand_data(universe, brands):
    """
    Pulls company specific data before the need to invoke segment level
    information. E.g., Bloomberg Second Measure Y/Y Sales, Days into 
    Quarter, Company Name, BICS Industry etc.
    """
    # Time series of daily transaction data
    bsm_daily_sales_online = (
        bq.data.observed_sales(dates=bq.func.range('-545D', '0D'), channel='Online')
        .dropna()
    )

    bsm_daily_sales_in_store = (
        bq.data.observed_sales(dates=bq.func.range('-545D', '0D'), channel='In_Store')
        .dropna()
    )

    # Reported quarter end date
    current_q_start = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='0'
    )['PERIOD_END_DATE']
    
    # Unreported quarter end date
    current_q_end = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='1'
    )['PERIOD_END_DATE']
    
    # Comparison quarter start date
    comparison_q_start = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='-4'
    )['PERIOD_END_DATE']
    
    # Comparison quarter end date
    comparison_q_end = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='-3'
    )['PERIOD_END_DATE']
    
    # The sum of the daily transactions across the elapsed days in the 
    # current quarter
    current_q_sales_online = (
        bsm_daily_sales_online
        .matches(
            (bsm_daily_sales_online['DATE'] > current_q_start)
            .and_(bsm_daily_sales_online['DATE'] <= current_q_end)
        )
    )

    current_q_sales_in_store = (
        bsm_daily_sales_in_store
        .matches(
            (bsm_daily_sales_in_store['DATE'] > current_q_start)
            .and_(bsm_daily_sales_in_store['DATE'] <= current_q_end)
        )
    )

    total_days_in_current_quarter = current_q_end - current_q_start
    
    # How many days into the current quarter do we have daily transaction 
    # data for, ie., how many days have elapsed since the last quarter end 
    # date, ensuring that if we have passed the expected current quarter end 
    # date, we cap at the total number of days in the quarter
    days_elapsed_in_current_q = bq.func.min(
        bsm_daily_sales_online.last()['DATE'] - current_q_start,
        total_days_in_current_quarter
    )
    
    # Isolate the comparison quarter sales figures to just be between the
    # comparison quarter start and end dates
    comparison_q_sales_online = (
        bsm_daily_sales_online
        .matches(
            (bsm_daily_sales_online['DATE'] > comparison_q_start)
            .and_(bsm_daily_sales_online['DATE'] <= comparison_q_end)
        )
    )

    comparison_q_sales_in_store = (
        bsm_daily_sales_in_store
        .matches(
            (bsm_daily_sales_in_store['DATE'] > comparison_q_start)
            .and_(bsm_daily_sales_in_store['DATE'] <= comparison_q_end)
        )
    )
    
    data_items = {
        'Total Days in Current Quarter': total_days_in_current_quarter,
        'Days Elapsed in Current Quarter': days_elapsed_in_current_q,
        'Days in Comp Quarter': comparison_q_end - comparison_q_start,
        'Current Quarter Online Sales': current_q_sales_online,
        'Current Quarter In-Store Sales': current_q_sales_in_store,
        'Comparison Quarter Online Sales': comparison_q_sales_online,
        'Comparison Quarter In-Store Sales': comparison_q_sales_in_store
    }
    
    with_params = {
        'act_est_mapping': 'precise',
        'filing_status': 'MRC'
    }

    request = bql.Request(universe, data_items, with_params=with_params, preferences=None)
    company_data = bq.execute(request)

    quarter_metadata = pd.concat([company_data[0].df(), company_data[1].df(),company_data[2].df()],axis=1).reset_index(drop=True)

    total_curr = quarter_metadata.loc[0,'Total Days in Current Quarter']
    days_in = quarter_metadata.loc[0,'Days Elapsed in Current Quarter']
    total_comp = quarter_metadata.loc[0,'Days in Comp Quarter']

    curr_q_online = company_data[3].df().reset_index(drop=True)[['PERIOD_END_DATE','Current Quarter Online Sales']]
    curr_q_in_store = company_data[4].df().reset_index(drop=True)[['Current Quarter In-Store Sales']]

    comp_q_online = company_data[5].df().reset_index(drop=True)[['Comparison Quarter Online Sales']]
    comp_q_in_store = company_data[6].df().reset_index(drop=True)[['Comparison Quarter In-Store Sales']]

    if days_in != total_curr:
        comp_q_online = comp_q_online.loc[:len(curr_q_online)-1]
        comp_q_in_store = comp_q_in_store.loc[:len(curr_q_online)-1]

    final_df = (
        pd.concat([curr_q_online,curr_q_in_store,comp_q_online,comp_q_in_store],axis=1)
        .fillna(0)
        .assign(curr_o_cum_sum = lambda x: x['Current Quarter Online Sales'].cumsum())
        .assign(comp_o_cum_sum = lambda x: x['Comparison Quarter Online Sales'].cumsum())
        .assign(qtd_online = lambda x: x['curr_o_cum_sum'] / x['comp_o_cum_sum'] - 1)
        
        .assign(curr_r_cum_sum = lambda x: x['Current Quarter In-Store Sales'].cumsum())
        .assign(comp_r_cum_sum = lambda x: x['Comparison Quarter In-Store Sales'].cumsum())
        .assign(qtd_in_store = lambda x: x['curr_r_cum_sum'] / x['comp_r_cum_sum'] - 1)
    
        .assign(qtd_total = lambda x: (x['curr_r_cum_sum'] +  x['curr_o_cum_sum']) / (x['comp_r_cum_sum'] + x['comp_o_cum_sum']) -1)
    
        .assign(pct_o = lambda x: x['curr_o_cum_sum'] / (x['curr_o_cum_sum'] + x['curr_r_cum_sum']))
        .assign(pct_r = lambda x: 1 - x['pct_o'])
    
        .rename(columns = {'qtd_online':'QTD - Online','qtd_in_store':'QTD - In-Store','qtd_total':'QTD - All','pct_o':'QTD - % Online','pct_r':'QTD - % In-Store'})
    )

    return final_df, quarter_metadata
    
#final_df = get_company_data(universe)

In [9]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.express as px
import time

# Assume these are already defined:
# possible_tickers, get_company_data(universe), get_transaction_data(universe), public_brands

# Global widgets for existing app.
status = widgets.HTML(value='')       # Spinner/status widget.
top_out = widgets.Output()            # For quarter progress HTML widget.
left_out = widgets.Output()           # For left column charts.
right_out = widgets.Output()          # For right column charts.

# New widgets for breakout selection.
breakout_dropdown = widgets.Dropdown(
    options=['Channel', 'Brands'],
    description='Breakout By:',
    value='Channel'
)
breakout_out = widgets.Output()       # For displaying brand checkboxes when "Brands" is selected.

# Global variables to store the brand checkboxes and selected brands.
brand_checkboxes = []
selected_brands = []

def update_breakout(ticker):
    """Update the breakout output widget based on breakout_dropdown.
       If 'Brands' is selected, filter public_brands for the given ticker and display
       checkboxes arranged in columns (max 5 rows per column). All checkboxes are initially selected.
       The global variable 'selected_brands' is updated with the list of selected brand names."""
    global brand_checkboxes, selected_brands
    with breakout_out:
        clear_output(wait=True)
        if breakout_dropdown.value == 'Brands':
            # Filter public_brands for the selected ticker and get unique brands.
            brands = public_brands[public_brands['Ticker'] == ticker]['Brand'].unique()
            # Create a list of checkboxes (one per brand), all selected by default.
            brand_checkboxes = [widgets.Checkbox(value=True, description=brand) for brand in brands]
            
            # Define a callback to update selected_brands when any checkbox changes.
            def update_selected_brands(change):
                global selected_brands
                selected_brands = [cb.description for cb in brand_checkboxes if cb.value]
                # Optionally, print the current list.
                # print("Selected Brands:", selected_brands)
            
            # Attach the observer to each checkbox.
            for cb in brand_checkboxes:
                cb.observe(update_selected_brands, names='value')
            
            # Helper function to chunk the list into groups of 5.
            def chunks(lst, n):
                return [lst[i:i+n] for i in range(0, len(lst), n)]
            
            # Create a list of VBox containers—each contains up to 5 checkboxes.
            cols = [widgets.VBox(chunk) for chunk in chunks(brand_checkboxes, 5)]
            # Arrange the columns side by side in an HBox.
            grid = widgets.HBox(cols)
            display(grid)
            # Initialize selected_brands based on default values.
            selected_brands = [cb.description for cb in brand_checkboxes if cb.value]



def update_plots(ticker):
    universe = ticker
    # Get the data.
    final_df, quarter_metadata = get_company_data(universe)
    # Drop duplicate "PERIOD_END_DATE" columns if they exist.
    final_df = final_df.loc[:, ~final_df.columns.duplicated()]
    together_total, together_pct = get_transaction_data(universe)
    
    # Create the left column figures.
    fig1 = px.line(
        final_df[7:],
        x='PERIOD_END_DATE',
        y=['QTD - Online', 'QTD - In-Store', 'QTD - All'],
        template='plotly_dark',
        title='QTD By Channel',
        labels={'value': 'Y/Y Observed Sales', 'variable': ''}
    )
    fig1.update_layout(yaxis=dict(tickformat=".1%"))
    fig1.update_xaxes(title_text="Days into Quarter")
    
    fig2 = px.area(
        final_df[7:],
        x='PERIOD_END_DATE',
        y=['QTD - % Online', 'QTD - % In-Store'],
        template='plotly_dark',
        title='QTD Share of Observed Sales',
        labels={'value': 'Share (%)', 'variable': ''}
    )
    fig2.update_layout(yaxis=dict(tickformat=".1%"))
    fig2.update_xaxes(title_text="Days into Quarter")
    
    # Create the right column figures.
    fig3 = px.line(
        together_pct,
        x='PERIOD_END_DATE',
        y=['Online', 'In-Store', 'Total'],
        template='plotly_dark',
        title='Fiscal Quarter Y/Y Growth',
        labels={'value': 'Y/Y Observed Sales', 'variable': ''}
    )
    fig3.update_layout(yaxis=dict(tickformat=".1%"))
    fig3.update_xaxes(title_text="Fiscal Period End Date")
    
    fig4 = px.area(
        together_total,
        x='PERIOD_END_DATE',
        y=['pct_online', 'pct_in_store'],
        template='plotly_dark',
        title='Fiscal Quarterly Share of Observed Sales',
        labels={'value': 'Share (%)', 'variable': ''}
    )
    fig4.update_layout(yaxis=dict(tickformat=".1%"))
    fig4.update_xaxes(title_text="Fiscal Period End Date")
    
    # Function to rename legends and update "All" traces.
    def rename_legends(fig):
        for trace in fig.data:
            name_lower = trace.name.lower()
            if "online" in name_lower:
                trace.name = "Online"
                trace.legendgroup = "Online"
            elif "store" in name_lower:
                trace.name = "In-Store"
                trace.legendgroup = "In-Store"
            elif name_lower in ["qtd - all", "total"]:
                trace.name = "All"
                trace.legendgroup = "All"
                trace.update(line=dict(dash='dash', width=4, color='white'))
    rename_legends(fig1)
    rename_legends(fig2)
    rename_legends(fig3)
    rename_legends(fig4)
    
    # Build the quarter progress HTML widget using rounded values.
    row_val = quarter_metadata.iloc[0]
    progress_html = (
        f"Quarter Progress: {int(round(row_val['Days Elapsed in Current Quarter']))} / "
        f"{int(round(row_val['Total Days in Current Quarter']))}<br>"
        f"Comp Quarter Length: {int(round(row_val['Days in Comp Quarter']))}"
    )
    progress_widget = widgets.HTML(value=progress_html)
    
    # Update the top output with the progress HTML.
    with top_out:
        clear_output(wait=True)
        display(progress_widget)
    
    # Update left and right outputs with the corresponding figures.
    with left_out:
        clear_output(wait=True)
        fig1.show()
        fig2.show()
    with right_out:
        clear_output(wait=True)
        fig3.show()
        fig4.show()
    
    # Once finished, clear the status spinner.
    status.value = ''
    
    # Update the breakout checkboxes if needed.
    update_breakout(ticker)

def on_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        status.value = '<div style="text-align:center;"><i class="fa fa-spinner fa-spin" style="font-size:24px;"></i> Loading...</div>'
        time.sleep(0.1)
        update_plots(change['new'])

def on_breakout_change(change):
    # When breakout selection changes, update the breakout checkboxes based on the current ticker.
    update_breakout(dropdown.value)

# Create and observe the ticker dropdown.
dropdown = widgets.Dropdown(
    options=possible_tickers,
    description='Ticker:',
    value='AAPL US Equity'
)
dropdown.observe(on_dropdown_change)

# Create and observe the breakout dropdown.
breakout_dropdown.observe(on_breakout_change)

# Wrap left and right outputs in VBoxes with a border.
left_box = widgets.VBox([left_out], layout=widgets.Layout(border='1px solid #CC5500', padding='5px', margin='5px'))
right_box = widgets.VBox([right_out], layout=widgets.Layout(border='1px solid #CC5500', padding='5px', margin='5px'))
columns = widgets.HBox([left_box, right_box])

# Display the widgets.
display(dropdown, breakout_dropdown, status, top_out, breakout_out, columns)

# Initialize with the default ticker.
update_plots(dropdown.value)

Dropdown(description='Ticker:', index=16, options=('139480 KS Equity', '1910 HK Equity', '3382 JP Equity', '43…

Dropdown(description='Breakout By:', options=('Channel', 'Brands'), value='Channel')

HTML(value='')

Output()

Output()

In [10]:
selected_brands 

[]